# Create all the necessary qmap files

All qmaps should be run in the ```./qmap_files/``` folder

## Prepare qmaps to get the depth per position
1. Samtools to get depth
2. Represent depth with an histogram
3. Annotate depth per mutation position

In [4]:
import json
import pandas as pd
import os

In [5]:
samples_dict = json.load( open( "../../../cases_ids.json", "rb" ) )
samples_dict

{'case1': {'normal': 'AQ5175',
  'tumor1': 'AQ5181',
  'tumor2': 'AQ5187',
  'sex': 'female'},
 'case2': {'normal': 'AQ5176',
  'tumor1': 'AQ5182',
  'tumor2': 'AQ5188',
  'sex': 'male'},
 'case3': {'normal': 'AQ5174',
  'tumor1': 'AQ5180',
  'tumor2': 'AQ5186',
  'sex': 'female',
  'kidney': 'AX4954',
  'liver': 'AX4955',
  'pancreas': 'AX4956',
  'heart': 'AX4957',
  'clone1': 'AX4958',
  'clone2': 'AX4961',
  'mother': 'AW8063',
  'father': 'AW8064',
  'lung': 'AX4962',
  'medulla': 'AX4963',
  'spleen': 'AX4964',
  'brain': 'AX4965',
  'bma': 'AX4966'},
 'case4': {'normal': 'AW8061',
  'tumor1': 'AW8050',
  'tumor2': 'AW8051',
  'sex': 'female'}}

### Samtools to get depth

In [7]:
root_in_crams = '/path/to/crams/'
root_out = '../depth/'

In [14]:
chroms = ['chr'+str(n) for n in range(1,23)]
chroms = chroms + ['chrX','chrY']

In [38]:
# make trees for storing the depth
samples = [ samples_dict[case][tissue] for tissue in samples_dict[case].keys() if tissue not in ['clone1','clone2','sex']]
samples

['AQ5174',
 'AQ5180',
 'AQ5186',
 'AX4954',
 'AX4955',
 'AX4956',
 'AX4957',
 'AW8063',
 'AW8064',
 'AX4962',
 'AX4963',
 'AX4964',
 'AX4965',
 'AX4966']

In [39]:
for sample in samples:
    folder = root_out + sample
    if not os.path.exists(folder):
        os.mkdir(folder)

In [48]:
commands = []

case = 'case3'
for tissue in samples_dict[case].keys():
    if tissue not in ['clone1','clone2','sex']:
        for chrom in chroms:
            sample_id = samples_dict[case][tissue]
            in_file = os.path.join(root_in_crams,sample_id+'.recal.cram')
            out_file = os.path.join(root_out,sample_id,chrom+'_depth.txt')
            command = 'samtools depth -r ' + chrom + ' ' + in_file + ' > ' + out_file
            commands.append(command)
commands

['samtools depth -r chr1 /path/to/crams/AQ5174.recal.cram > ../depth/AQ5174/chr1_depth.txt',
 'samtools depth -r chr2 /path/to/crams/AQ5174.recal.cram > ../depth/AQ5174/chr2_depth.txt',
 'samtools depth -r chr3 /path/to/crams/AQ5174.recal.cram > ../depth/AQ5174/chr3_depth.txt',
 'samtools depth -r chr4 /path/to/crams/AQ5174.recal.cram > ../depth/AQ5174/chr4_depth.txt',
 'samtools depth -r chr5 /path/to/crams/AQ5174.recal.cram > ../depth/AQ5174/chr5_depth.txt',
 'samtools depth -r chr6 /path/to/crams/AQ5174.recal.cram > ../depth/AQ5174/chr6_depth.txt',
 'samtools depth -r chr7 /path/to/crams/AQ5174.recal.cram > ../depth/AQ5174/chr7_depth.txt',
 'samtools depth -r chr8 /path/to/crams/AQ5174.recal.cram > ../depth/AQ5174/chr8_depth.txt',
 'samtools depth -r chr9 /path/to/crams/AQ5174.recal.cram > ../depth/AQ5174/chr9_depth.txt',
 'samtools depth -r chr10 /path/to/crams/AQ5174.recal.cram > ../depth/AQ5174/chr10_depth.txt',
 'samtools depth -r chr11 /path/to/crams/AQ5174.recal.cram > ../dept

In [49]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate samtools','[params]','cores = 1','memory = 8G','[jobs]']
qmap_file = qmap_pre_params + commands

In [50]:
#Save qmap file

with open('./qmap_files/01_samtools_depth_per_chromosome.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

### Calculate histogram and stats from depth

In [51]:
#commands for process vcfs from strelka

python_file = '../scripts/depth_stats_histogram.py'

commands = []

case = 'case3'
for tissue in samples_dict[case].keys():
    if tissue not in ['clone1','clone2','sex']:
        for chrom in chroms:
            sample_id = samples_dict[case][tissue]
            in_file = os.path.join(root_out,sample_id,chrom+'_depth.txt')
            out_file_stats = os.path.join(root_out,sample_id,chrom+'_stats.json')
            out_file_hist = os.path.join(root_out,sample_id,chrom+'_hist.png')
            command = 'python ' + python_file + ' --input_file ' + in_file + ' --output_file_stats ' + out_file_stats + ' --output_file_hist ' + out_file_hist
            commands.append(command)
commands

['python ../scripts/depth_stats_histogram.py --input_file ../depth/AQ5174/chr1_depth.txt --output_file_stats ../depth/AQ5174/chr1_stats.json --output_file_hist ../depth/AQ5174/chr1_hist.png',
 'python ../scripts/depth_stats_histogram.py --input_file ../depth/AQ5174/chr2_depth.txt --output_file_stats ../depth/AQ5174/chr2_stats.json --output_file_hist ../depth/AQ5174/chr2_hist.png',
 'python ../scripts/depth_stats_histogram.py --input_file ../depth/AQ5174/chr3_depth.txt --output_file_stats ../depth/AQ5174/chr3_stats.json --output_file_hist ../depth/AQ5174/chr3_hist.png',
 'python ../scripts/depth_stats_histogram.py --input_file ../depth/AQ5174/chr4_depth.txt --output_file_stats ../depth/AQ5174/chr4_stats.json --output_file_hist ../depth/AQ5174/chr4_hist.png',
 'python ../scripts/depth_stats_histogram.py --input_file ../depth/AQ5174/chr5_depth.txt --output_file_stats ../depth/AQ5174/chr5_stats.json --output_file_hist ../depth/AQ5174/chr5_hist.png',
 'python ../scripts/depth_stats_histogra

In [52]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 1','memory = 50G','[jobs]']
qmap_file = qmap_pre_params + commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 1',
 'memory = 50G',
 '[jobs]',
 'python ../scripts/depth_stats_histogram.py --input_file ../depth/AQ5174/chr1_depth.txt --output_file_stats ../depth/AQ5174/chr1_stats.json --output_file_hist ../depth/AQ5174/chr1_hist.png',
 'python ../scripts/depth_stats_histogram.py --input_file ../depth/AQ5174/chr2_depth.txt --output_file_stats ../depth/AQ5174/chr2_stats.json --output_file_hist ../depth/AQ5174/chr2_hist.png',
 'python ../scripts/depth_stats_histogram.py --input_file ../depth/AQ5174/chr3_depth.txt --output_file_stats ../depth/AQ5174/chr3_stats.json --output_file_hist ../depth/AQ5174/chr3_hist.png',
 'python ../scripts/depth_stats_histogram.py --input_file ../depth/AQ5174/chr4_depth.txt --output_file_stats ../depth/AQ5174/chr4_stats.json --output_file_hist ../depth/AQ5174/chr4_hist.png',
 'python ../scripts/depth_stats_histogram.py --input_file ../depth/AQ5174/chr5_dept

In [53]:
#Save qmap file

with open('./qmap_files/02_histogram_and_stats_from_depth.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

### Annotate depth per mutation position

In [12]:
#commands for process vcfs

python_file = '../scripts/depth_from_muts.py'

commands = []

case = 'case3'
for tissue in samples_dict[case].keys():
    if tissue not in ['clone1','clone2','sex']:
        sample_id = samples_dict[case][tissue]
        in_somatic_muts = '../output/01_somatic_from_hc_mutect_parents.tsv'
        in_folder_depth = root_out+sample_id+'/'
        out_file = os.path.join(root_out,sample_id+'_depth_muts_hc_mutect_parents.tsv.gz')
        command = 'python ' + python_file + ' --input_somatic_muts ' + in_somatic_muts + ' --input_folder ' + in_folder_depth + ' --output_file ' + out_file
        commands.append(command)
commands

['python ../scripts/depth_from_muts.py --input_somatic_muts ../output/01_somatic_from_hc_mutect_parents.tsv --input_folder ../depth/AQ5174/ --output_file ../depth/AQ5174_depth_muts_hc_mutect_parents.tsv.gz',
 'python ../scripts/depth_from_muts.py --input_somatic_muts ../output/01_somatic_from_hc_mutect_parents.tsv --input_folder ../depth/AQ5180/ --output_file ../depth/AQ5180_depth_muts_hc_mutect_parents.tsv.gz',
 'python ../scripts/depth_from_muts.py --input_somatic_muts ../output/01_somatic_from_hc_mutect_parents.tsv --input_folder ../depth/AQ5186/ --output_file ../depth/AQ5186_depth_muts_hc_mutect_parents.tsv.gz',
 'python ../scripts/depth_from_muts.py --input_somatic_muts ../output/01_somatic_from_hc_mutect_parents.tsv --input_folder ../depth/AX4954/ --output_file ../depth/AX4954_depth_muts_hc_mutect_parents.tsv.gz',
 'python ../scripts/depth_from_muts.py --input_somatic_muts ../output/01_somatic_from_hc_mutect_parents.tsv --input_folder ../depth/AX4955/ --output_file ../depth/AX495

In [13]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 1','memory = 50G','[jobs]']
qmap_file = qmap_pre_params + commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 1',
 'memory = 50G',
 '[jobs]',
 'python ../scripts/depth_from_muts.py --input_somatic_muts ../output/01_somatic_from_hc_mutect_parents.tsv --input_folder ../depth/AQ5174/ --output_file ../depth/AQ5174_depth_muts_hc_mutect_parents.tsv.gz',
 'python ../scripts/depth_from_muts.py --input_somatic_muts ../output/01_somatic_from_hc_mutect_parents.tsv --input_folder ../depth/AQ5180/ --output_file ../depth/AQ5180_depth_muts_hc_mutect_parents.tsv.gz',
 'python ../scripts/depth_from_muts.py --input_somatic_muts ../output/01_somatic_from_hc_mutect_parents.tsv --input_folder ../depth/AQ5186/ --output_file ../depth/AQ5186_depth_muts_hc_mutect_parents.tsv.gz',
 'python ../scripts/depth_from_muts.py --input_somatic_muts ../output/01_somatic_from_hc_mutect_parents.tsv --input_folder ../depth/AX4954/ --output_file ../depth/AX4954_depth_muts_hc_mutect_parents.tsv.gz',
 'python ../scripts

In [14]:
#Save qmap file

with open('./qmap_files/03_annotate_depth_per_mut_pos.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

-------------------------------------------

## qmaps to annotate varianst with vep

### Run vep on somatic mutations

In [9]:
chroms = list(range(1,23))
chroms = ['chr'+str(chrom) for chrom in chroms]
sex_chroms = ['chrX','chrY']
chroms = chroms + sex_chroms

mafs_url = '../vep/vep_input/'

gnomad_url = '/path/to/gnomad3.0/'

In [10]:
#run vep command

vep_commands = []
for chrom in chroms:
    pt_url = os.path.join(mafs_url, chrom + '_somatic_muts.tsv.gz')
    chr_file = 'gnomad.genomes.r3.0.sites.'+chrom+'.vcf.bgz'
    gnomad_chr_file = os.path.join(gnomad_url,chr_file)
    out_file = pt_url.replace('vep_input/', 'vep_output/')
    out_file = out_file.replace('.tsv.gz', '_vep.tsv')
    vep_command = 'vep -i '+ pt_url + ' -o ' + out_file + ' -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /workspace/datasets/vep  --custom ' + gnomad_chr_file + ',gnomADg,vcf,exact,0,AF,NFE'
    vep_commands.append(vep_command)
        
vep_commands

['vep -i ../vep/vep_input/chr1_somatic_muts.tsv.gz -o ../vep/vep_output/chr1_somatic_muts_vep.tsv -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /workspace/datasets/vep  --custom /path/to/gnomad3.0/gnomad.genomes.r3.0.sites.chr1.vcf.bgz,gnomADg,vcf,exact,0,AF,NFE',
 'vep -i ../vep/vep_input/chr2_somatic_muts.tsv.gz -o ../vep/vep_output/chr2_somatic_muts_vep.tsv -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /workspace/datasets/vep  --custom /path/to/gnomad3.0/gnomad.genomes.r3.0.sites.chr2.vcf.bgz,gnomADg,vcf,exact,0,AF,NFE',
 'vep -i ../vep/vep_input/chr3_somatic_muts.tsv.gz -o ../vep/vep_output/chr3_somatic_muts_vep.tsv -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /workspace/datasets/vep  --custom /path/to/gnomad3.0/gnomad.genomes.r3.0.sites.chr3.vcf.bgz,gnomADg,vcf,exact,0,AF,NFE',
 'vep -i ../vep/vep_input/chr4_somatic_muts.tsv.g

In [11]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate vep101','[params]','cores = 1','memory = 8G','[jobs]']
qmap_file = qmap_pre_params + vep_commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate vep101',
 '[params]',
 'cores = 1',
 'memory = 8G',
 '[jobs]',
 'vep -i ../vep/vep_input/chr1_somatic_muts.tsv.gz -o ../vep/vep_output/chr1_somatic_muts_vep.tsv -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /workspace/datasets/vep  --custom /path/to/gnomad3.0/gnomad.genomes.r3.0.sites.chr1.vcf.bgz,gnomADg,vcf,exact,0,AF,NFE',
 'vep -i ../vep/vep_input/chr2_somatic_muts.tsv.gz -o ../vep/vep_output/chr2_somatic_muts_vep.tsv -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /workspace/datasets/vep  --custom /path/to/gnomad3.0/gnomad.genomes.r3.0.sites.chr2.vcf.bgz,gnomADg,vcf,exact,0,AF,NFE',
 'vep -i ../vep/vep_input/chr3_somatic_muts.tsv.gz -o ../vep/vep_output/chr3_somatic_muts_vep.tsv -tab --assembly GRCh38 --no_stats --cache --symbol --protein --canonical --offline --af_1kg --dir /workspace/datasets/vep  -

In [12]:
with open('./qmap_files/04_run_vep_gnomad_v3_somatic_muts.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

### Process vep

In [17]:
#run process_vep command
python_script = '../scripts/process_vep101_alltissues.py'
process_vep_commands = []
for chrom in chroms:
    input_url = '../vep/vep_output/'+chrom+'_somatic_muts_vep.tsv'
    output_url = '../vep/vep_process/'+chrom+'_somatic_muts_vep_filt.tsv.gz'
    command = 'python '+python_script+' --input '+input_url+' --output '+output_url+' --cores 8'
    process_vep_commands.append(command)    
    
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate process_vc','[params]','cores = 8','memory = 15G','[jobs]']
qmap_file = qmap_pre_params + process_vep_commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate process_vc',
 '[params]',
 'cores = 8',
 'memory = 15G',
 '[jobs]',
 'python ../scripts/process_vep101_alltissues.py --input ../vep/vep_output/chr1_somatic_muts_vep.tsv --output ../vep/vep_process/chr1_somatic_muts_vep_filt.tsv.gz --cores 8',
 'python ../scripts/process_vep101_alltissues.py --input ../vep/vep_output/chr2_somatic_muts_vep.tsv --output ../vep/vep_process/chr2_somatic_muts_vep_filt.tsv.gz --cores 8',
 'python ../scripts/process_vep101_alltissues.py --input ../vep/vep_output/chr3_somatic_muts_vep.tsv --output ../vep/vep_process/chr3_somatic_muts_vep_filt.tsv.gz --cores 8',
 'python ../scripts/process_vep101_alltissues.py --input ../vep/vep_output/chr4_somatic_muts_vep.tsv --output ../vep/vep_process/chr4_somatic_muts_vep_filt.tsv.gz --cores 8',
 'python ../scripts/process_vep101_alltissues.py --input ../vep/vep_output/chr5_somatic_muts_vep.tsv --output ../vep/vep_process/chr5_somatic_muts_vep_

In [18]:
with open('./qmap_files/05_process_vep.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)

---------------------------------

## qmap to calculate betabinomial function, to detect artifacst

In [25]:
python_file = '../scripts/calculate_pvals_betabinom.py'

chroms = ['chr'+str(n) for n in range(1,23)]
chroms = chroms + ['chrX', 'chrY']

betabinom_path = '../betabinom/'
suffixes_json = '../dictionaries/suffixes_dict.json'
suffixes_blood_json = '../dictionaries/blood_suffixes_dict.json'

commands = []

for chrom in chroms:
    input_file = betabinom_path + chrom + '_input.tsv.gz'
    output_file = betabinom_path + chrom + '_output.tsv.gz'
    command = 'python '+python_file+' --muts_file '+input_file+' --suffixes_json '+suffixes_json+' --suffixes_blood_json '+suffixes_blood_json+' --tolerance 30 --output_file '+output_file+' --chunk_size 100'
    commands.append(command)
commands

['python ../scripts/calculate_pvals_betabinom.py --muts_file ../betabinom/chr1_input.tsv.gz --suffixes_json ../dictionaries/suffixes_dict.json --suffixes_blood_json ../dictionaries/blood_suffixes_dict.json --tolerance 30 --output_file ../betabinom/chr1_output.tsv.gz --chunk_size 100',
 'python ../scripts/calculate_pvals_betabinom.py --muts_file ../betabinom/chr2_input.tsv.gz --suffixes_json ../dictionaries/suffixes_dict.json --suffixes_blood_json ../dictionaries/blood_suffixes_dict.json --tolerance 30 --output_file ../betabinom/chr2_output.tsv.gz --chunk_size 100',
 'python ../scripts/calculate_pvals_betabinom.py --muts_file ../betabinom/chr3_input.tsv.gz --suffixes_json ../dictionaries/suffixes_dict.json --suffixes_blood_json ../dictionaries/blood_suffixes_dict.json --tolerance 30 --output_file ../betabinom/chr3_output.tsv.gz --chunk_size 100',
 'python ../scripts/calculate_pvals_betabinom.py --muts_file ../betabinom/chr4_input.tsv.gz --suffixes_json ../dictionaries/suffixes_dict.json

In [28]:
qmap_pre_params = ['[pre]','. "/home/$USER/miniconda3/etc/profile.d/conda.sh"','conda activate tensorflow','[params]','cores = 56','memory = 200G','[jobs]']
qmap_file = qmap_pre_params + commands
qmap_file

['[pre]',
 '. "/home/$USER/miniconda3/etc/profile.d/conda.sh"',
 'conda activate tensorflow',
 '[params]',
 'cores = 56',
 'memory = 200G',
 '[jobs]',
 'python ../scripts/calculate_pvals_betabinom.py --muts_file ../betabinom/chr1_input.tsv.gz --suffixes_json ../dictionaries/suffixes_dict.json --suffixes_blood_json ../dictionaries/blood_suffixes_dict.json --tolerance 30 --output_file ../betabinom/chr1_output.tsv.gz --chunk_size 100',
 'python ../scripts/calculate_pvals_betabinom.py --muts_file ../betabinom/chr2_input.tsv.gz --suffixes_json ../dictionaries/suffixes_dict.json --suffixes_blood_json ../dictionaries/blood_suffixes_dict.json --tolerance 30 --output_file ../betabinom/chr2_output.tsv.gz --chunk_size 100',
 'python ../scripts/calculate_pvals_betabinom.py --muts_file ../betabinom/chr3_input.tsv.gz --suffixes_json ../dictionaries/suffixes_dict.json --suffixes_blood_json ../dictionaries/blood_suffixes_dict.json --tolerance 30 --output_file ../betabinom/chr3_output.tsv.gz --chunk_si

In [29]:
with open('./qmap_files/06_calculate_betabinom.qmap', 'w') as f:
    for item in qmap_file:
        f.write('%s\n' % item)